# 1. 特征提取
使用ResNet18 -> 512维特征向量
可以在Grok里回顾 （搜索：“我有几千张照片，我说100张，是为了先运行一遍完整的流程”

In [1]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os
import numpy as np

# 加载ResNet18
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = models.resnet18(pretrained=True).to(device)
model = torch.nn.Sequential(*list(model.children())[:-1])  # 移除分类层
model.eval()

/Users/xurui/miniforge3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/xurui/miniforge3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

图像预处理

In [ ]:
# 图像预处理
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 特征提取函数（支持批量）
def extract_features_batch(image_paths, batch_size=32):
    features_list = []
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i + batch_size]
        batch_images = [preprocess(Image.open(p).convert("RGB")) for p in batch_paths]
        batch_tensor = torch.stack(batch_images).to(device)
        with torch.no_grad():
            batch_features = model(batch_tensor)  # [batch_size, 512, 1, 1]
            batch_features = batch_features.squeeze(-1).squeeze(-1)  # [batch_size, 512]
        features_list.append(batch_features.cpu().numpy())
    return np.vstack(features_list)

# 处理图片
image_dir = "path/to/your/images"
image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) 
               if img.endswith((".jpg", ".png"))]
image_paths = image_paths[:100]  # 先跑100张

# 提取特征
image_features = extract_features_batch(image_paths, batch_size=32)  # shape: (100, 512)
np.save("image_features.npy", image_features)  # 保存特征

# 2. 拼接 CNV 向量

In [ ]:
# 示例：其他向量（替换为你的真实数据）
n_samples = len(image_paths)
other_vectors = np.random.rand(n_samples, 10)  # shape: (100, 10)

# 拼接特征
combined_features = np.hstack((image_features, other_vectors))  # shape: (100, 522)

# 3. 训练逻辑回归模型

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 示例标签（替换为你的真实标签）
binary_labels = np.random.randint(0, 2, n_samples)  # shape: (100,)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    combined_features, binary_labels, test_size=0.2, random_state=42
)

# 训练逻辑回归
clf = LogisticRegression(max_iter=1000, penalty='l2', C=1.0)
clf.fit(X_train, y_train)

# 预测与评估
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# 保存模型
import joblib
joblib.dump(clf, "logistic_model.pkl")